# Segmenting and Clustering

This notebook will be used for the overall assignment "Segmenting and Clustering Neighborhoods in Toronto".

## Importing libraries

We import immediatly the two most important libraries for data science.

In [1]:
import pandas as pd
import numpy as np

## Importing data

We import data from [Wikipedia][1] about the neighborhood of Toronto with *pandas* method. Then, we convert the data into a *pandas* dataframe to be able to use the full potential of this librarie.

[1]: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [2]:
#Importing data
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df = pd.read_html(url)[0]
list_reject = list()

#Precleaning of the data
for i in range(len(df)):
    if df.loc[i, 'Borough'] == 'Not assigned':
        list_reject.append(i)
    elif df.loc[i, 'Neighbourhood'] == 'Not assigned':
        df.loc[i, 'Neighbourhood'] = df.loc[i, 'Borough']

df.drop(list_reject, inplace = True)
df.reset_index(inplace = True)
df.drop(['index'], axis=1, inplace = True)


In the following cell, we are trying to verify if the previous dataframe is the one we want.

In [3]:
for i in range(len(df)):
    if df.loc[i, 'Borough'] == 'Not assigned' or df.loc[i, 'Neighbourhood'] == 'Not assigned':
        print("Still a 'Not assigned' value")
        break
print("No 'Not assigned' value remaining")

if df['Postal Code'].value_counts()[0] == 1:
    print("Any two different rows have different postal codes")
else:
    print("Issue on postal codes")
    
print("Shape of the dataframe : ", df.shape)

No 'Not assigned' value remaining
Any two different rows have different postal codes
Shape of the dataframe :  (103, 3)


## Adding informations to the dataframe

Since *geocoder* package doesn't work, we will use the csv file provided [here][1] to complete our dataframe.

[1]: https://cocl.us/Geospatial_data

In [4]:
geo_path = 'Data\Geospatial_Coordinates.csv'
df_geo = pd.read_csv(geo_path)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Then, we merge the two previous dataframes on "Postal Code" to obtain the final dataframe.

In [5]:
df_tor = pd.merge(df, df_geo, on="Postal Code")
df_tor.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [6]:
df_tor.dtypes

Postal Code       object
Borough           object
Neighbourhood     object
Latitude         float64
Longitude        float64
dtype: object

We can use *folium* package to have a first map of the city of Toronto.

In [7]:
import folium

lat_tor = 43.7
lon_tor = -79.4

tor_map = folium.Map(location = [lat_tor, lon_tor], zoom_start=11)
tor_map

Then, we add markers to indicate locations of all postal code across the city. We use colors to distinguish the different boroughs.

In [8]:
import matplotlib.cm as cm
import matplotlib.colors as colors

num_bor = len(pd.unique(df_tor["Borough"]))
color_list = cm.rainbow(np.linspace(0,1,num_bor))
rainbow = [colors.rgb2hex(i) for i in color_list]

tor_map = folium.Map(location = [lat_tor, lon_tor], zoom_start=11)

marker_color = {}
i=-1

for lat, lon, code, bor in zip(df_tor["Latitude"], df_tor["Longitude"], df_tor["Postal Code"], df_tor["Borough"]):
    label = folium.Popup(code +', '+ bor, parse_html=True)
    try :
        col = marker_color[str(bor)]
    except :
        i+=1
        marker_color[str(bor)]=i
            
    folium.CircleMarker([lat, lon], radius =5, popup = label, color = rainbow[marker_color[str(bor)]], fill =True).add_to(tor_map)

tor_map

## Exploring and clustering the neighborhoods in Toronto

First, we will begin to gather informations on the neighborhoods using the Foursquare API. We must define our Foursquare API credentials to be able to use it.

In [9]:
#Define my credentials and version

CLIENT_ID = 'SLUHRYUL0LL0TTU1XEBXTMRF2VEKGSNVOPJ24T3MAKXDT1II'
CLIENT_SECRET = 'QDE5KQAESJOKO1YRLB4UW4MVC0XIXJBCMXRJY4FUW4RJNS4Q'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

Then, we make several requests to have informations on eache neighborhoods. To do not exceed the maximum request we can made in a day with a free developer account, we will work only on neighborhoods containing *'Toronto'* in the borough's name:
* Downtown Toronto
* East Toronto
* West Toronto
* Central Toronto

In [15]:
#Get the best venues around each neighborhood
import json
import requests

def getVenues(code, borough, latitude, longitude, radius=500):

    nearby_venues = pd.DataFrame(columns = ["Postal Code", "Borough", "PC_lat", "PC_lon", "Venue_name", "Venue_categorie"])
    i=0
    
    for code, borough, lat, lon in zip(code, borough, latitude, longitude):
        if 'Toronto' in str(borough):
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lon, 
                radius, 
                LIMIT)

            print(i)
            results = requests.get(url).json()
            print(results)
            results = results["response"]['groups'][0]['items']


            for res in results:
                pc = code
                pc_lat = lat
                pc_lon = lon
                venue = res['venue']['name']
                cat = res['venue']['categories'][0]['name']

                nearby_venues.loc[i] = [pc, borough, pc_lat, pc_lon, venue, cat]
                i+=1
    
    return(nearby_venues)

In [16]:
venues = getVenues(df_tor['Postal Code'],df_tor['Borough'], df_tor['Latitude'], df_tor['Longitude'])
venues.head()

0
{'meta': {'code': 200, 'requestId': '5fca059d5b23bb2330b79204'}, 'response': {'headerLocation': 'Corktown', 'headerFullLocation': 'Corktown, Toronto', 'headerLocationGranularity': 'neighborhood', 'totalResults': 44, 'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486}, 'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '54ea41ad498e9a11e9e13308', 'name': 'Roselle Desserts', 'location': {'address': '362 King St E', 'crossStreet': 'Trinity St', 'lat': 43.653446723052674, 'lng': -79.3620167174383, 'labeledLatLngs': [{'label': 'display', 'lat': 43.653446723052674, 'lng': -79.3620167174383}], 'distance': 143, 'postalCode': 'M5A 1K9', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['362 King St E (Tri

{'meta': {'code': 200, 'requestId': '5fca059d4419a9763d81f026'}, 'response': {'headerLocation': 'Bay Street Corridor', 'headerFullLocation': 'Bay Street Corridor, Toronto', 'headerLocationGranularity': 'neighborhood', 'totalResults': 33, 'suggestedBounds': {'ne': {'lat': 43.6668015045, 'lng': -79.38328496866619}, 'sw': {'lat': 43.657801495499996, 'lng': -79.3957026313338}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4b9d206bf964a520e69136e3', 'name': "Queen's Park", 'location': {'address': 'University Ave.', 'crossStreet': 'at Wellesley Ave.', 'lat': 43.66394609897775, 'lng': -79.39217952520835, 'labeledLatLngs': [{'label': 'display', 'lat': 43.66394609897775, 'lng': -79.39217952520835}], 'distance': 283, 'postalCode': 'M5R 2E8', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formatted

{'meta': {'code': 200, 'requestId': '5fca059df853e9681ddfbf68'}, 'response': {'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}]}, 'headerLocation': 'Downtown Toronto', 'headerFullLocation': 'Downtown Toronto, Toronto', 'headerLocationGranularity': 'neighborhood', 'totalResults': 126, 'suggestedBounds': {'ne': {'lat': 43.6616618045, 'lng': -79.37272880013877}, 'sw': {'lat': 43.652661795499995, 'lng': -79.3851453998612}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '57eda381498ebe0e6ef40972', 'name': 'UNIQLO ユニクロ', 'location': {'address': '220 Yonge St', 'crossStreet': 'at Dundas St W', 'lat': 43.65591027779457, 'lng': -79.38064099181345, 'labeledLatLngs': [{'label': 'display', 'lat': 43.65591027779457, 'lng': -79.38064099181345}], 'distance': 195, 'postalCode':

177
{'meta': {'code': 200, 'requestId': '5fca059e6d31c0183b677700'}, 'response': {'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}]}, 'headerLocation': 'St. Lawrence', 'headerFullLocation': 'St. Lawrence, Toronto', 'headerLocationGranularity': 'neighborhood', 'totalResults': 85, 'suggestedBounds': {'ne': {'lat': 43.6559939045, 'lng': -79.36921018606671}, 'sw': {'lat': 43.646993895499996, 'lng': -79.3816256139333}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4b49183ff964a520a46526e3', 'name': 'Terroni', 'location': {'address': '57 Adelaide St. E', 'crossStreet': 'at Church St.', 'lat': 43.650927, 'lng': -79.375602, 'labeledLatLngs': [{'label': 'display', 'lat': 43.650927, 'lng': -79.375602}], 'distance': 64, 'postalCode': 'M5C 1K6', 'cc': 'CA', 'neighborhood'

262
{'meta': {'code': 200, 'requestId': '5fca059f7c1f474e474e4362'}, 'response': {'headerLocation': 'The Beaches', 'headerFullLocation': 'The Beaches, Toronto', 'headerLocationGranularity': 'neighborhood', 'totalResults': 4, 'suggestedBounds': {'ne': {'lat': 43.680857404499996, 'lng': -79.28682091449052}, 'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4bd461bc77b29c74a07d9282', 'name': 'Glen Manor Ravine', 'location': {'address': 'Glen Manor', 'crossStreet': 'Queen St.', 'lat': 43.67682094413784, 'lng': -79.29394208780985, 'labeledLatLngs': [{'label': 'display', 'lat': 43.67682094413784, 'lng': -79.29394208780985}], 'distance': 89, 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Glen Manor (Queen St.)', 'Toro

...

1385
{'meta': {'code': 200, 'requestId': '5fca05adf182532cb0beec4e'}, 'response': {'headerLocation': 'Cabbagetown', 'headerFullLocation': 'Cabbagetown, Toronto', 'headerLocationGranularity': 'neighborhood', 'totalResults': 48, 'suggestedBounds': {'ne': {'lat': 43.6724670045, 'lng': -79.3614658826597}, 'sw': {'lat': 43.663466995499995, 'lng': -79.3738847173403}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4b646a6ff964a5205cb12ae3', 'name': 'Cranberries', 'location': {'address': '601 Parliament St.', 'crossStreet': 'at Wellesley St. E', 'lat': 43.6678427705951, 'lng': -79.36940687874281, 'labeledLatLngs': [{'label': 'display', 'lat': 43.6678427705951, 'lng': -79.36940687874281}], 'distance': 140, 'postalCode': 'M4X 1P9', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': [

{'meta': {'code': 200, 'requestId': '5fca05aee0651225f9562e2c'}, 'response': {'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}]}, 'headerLocation': 'Financial District', 'headerFullLocation': 'Financial District, Toronto', 'headerLocationGranularity': 'neighborhood', 'totalResults': 161, 'suggestedBounds': {'ne': {'lat': 43.652929204500005, 'lng': -79.37607280281344}, 'sw': {'lat': 43.6439291955, 'lng': -79.38848759718655}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4ad4c05df964a52059f620e3', 'name': 'Canoe', 'location': {'address': '66 Wellington St West', 'crossStreet': 'at Bay Street', 'lat': 43.647452066183476, 'lng': -79.38132001815676, 'labeledLatLngs': [{'label': 'display', 'lat': 43.647452066183476, 'lng': -79.38132001815676}], 'distance': 133, 'pos

1533
{'meta': {'code': 200, 'requestId': '5fca05aee749390203860569'}, 'response': {'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}]}, 'headerLocation': 'The Village', 'headerFullLocation': 'The Village, Toronto', 'headerLocationGranularity': 'neighborhood', 'totalResults': 75, 'suggestedBounds': {'ne': {'lat': 43.6703599045, 'lng': -79.37695070062593}, 'sw': {'lat': 43.6613598955, 'lng': -79.3893690993741}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '5bd2379cbcbf7a0039a2d7b9', 'name': 'Storm Crow Manor', 'location': {'address': '580 Church St', 'lat': 43.66683985695023, 'lng': -79.38159278029707, 'labeledLatLngs': [{'label': 'display', 'lat': 43.66683985695023, 'lng': -79.38159278029707}], 'distance': 166, 'postalCode': 'M4Y 2E5', 'cc': 'CA', 'neighborhood'

1608
{'meta': {'code': 200, 'requestId': '5fca05afa788655b3e091050'}, 'response': {'headerLocation': 'Leslieville', 'headerFullLocation': 'Leslieville, Toronto', 'headerLocationGranularity': 'neighborhood', 'totalResults': 16, 'suggestedBounds': {'ne': {'lat': 43.667243904500005, 'lng': -79.31534912291306}, 'sw': {'lat': 43.6582438955, 'lng': -79.32776687708693}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '58d4650e2f91cb5ccec03447', 'name': 'Rorschach Brewing Co.', 'location': {'address': '1001 Eastern Ave', 'crossStreet': 'Woodfield Rd', 'lat': 43.6634831695922, 'lng': -79.31982368639481, 'labeledLatLngs': [{'label': 'display', 'lat': 43.6634831695922, 'lng': -79.31982368639481}], 'distance': 162, 'postalCode': 'M4L 1A8', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress

,Postal Code,Borough,PC_lat,PC_lon,Venue_name,Venue_categorie
0,M5A,Downtown Toronto,43.65426,-79.360636,Roselle Desserts,Bakery
1,M5A,Downtown Toronto,43.65426,-79.360636,Tandem Coffee,Coffee Shop
2,M5A,Downtown Toronto,43.65426,-79.360636,Cooper Koo Family YMCA,Distribution Center
3,M5A,Downtown Toronto,43.65426,-79.360636,Body Blitz Spa East,Spa
4,M5A,Downtown Toronto,43.65426,-79.360636,Impact Kitchen,Restaurant


In [17]:
venues.shape

(1624, 6)

In [19]:
venues.groupby("Postal Code").count()

,Borough,PC_lat,PC_lon,Venue_name,Venue_categorie
Postal Code,,,,,
M4E,4,4,4,4,4
M4K,43,43,43,43,43
M4L,19,19,19,19,19
M4M,37,37,37,37,37
M4N,3,3,3,3,3
M4P,9,9,9,9,9
M4R,18,18,18,18,18
M4S,33,33,33,33,33
M4T,2,2,2,2,2


In [40]:
venues_cat_sep = pd.get_dummies(venues[["Venue_categorie"]])
venues_cat_sep["Postal Code"] = venues["Postal Code"]

fixed_columns = [venues_cat_sep.columns[-1]] + list(venues_cat_sep.columns[:-1])
venues_cat_sep = venues_cat_sep[fixed_columns]

venues_cat_sep.head()

,Postal Code,Venue_categorie_Afghan Restaurant,Venue_categorie_Airport,Venue_categorie_Airport Food Court,Venue_categorie_Airport Gate,Venue_categorie_Airport Lounge,Venue_categorie_Airport Service,Venue_categorie_Airport Terminal,Venue_categorie_American Restaurant,Venue_categorie_Antique Shop,...,Venue_categorie_Theater,Venue_categorie_Theme Restaurant,Venue_categorie_Toy / Game Store,Venue_categorie_Trail,Venue_categorie_Train Station,Venue_categorie_Vegetarian / Vegan Restaurant,Venue_categorie_Video Game Store,Venue_categorie_Vietnamese Restaurant,Venue_categorie_Wine Bar,Venue_categorie_Yoga Studio
0,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
venues_grouped = venues_cat_sep.groupby("Postal Code").sum()
venues_grouped 

,Venue_categorie_Afghan Restaurant,Venue_categorie_Airport,Venue_categorie_Airport Food Court,Venue_categorie_Airport Gate,Venue_categorie_Airport Lounge,Venue_categorie_Airport Service,Venue_categorie_Airport Terminal,Venue_categorie_American Restaurant,Venue_categorie_Antique Shop,Venue_categorie_Aquarium,...,Venue_categorie_Theater,Venue_categorie_Theme Restaurant,Venue_categorie_Toy / Game Store,Venue_categorie_Trail,Venue_categorie_Train Station,Venue_categorie_Vegetarian / Vegan Restaurant,Venue_categorie_Video Game Store,Venue_categorie_Vietnamese Restaurant,Venue_categorie_Wine Bar,Venue_categorie_Yoga Studio
Postal Code,,,,,,,,,,,,,,,,,,,,,
M4E,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
M4K,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
M4L,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M4M,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,1,1
M4N,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M4P,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M4R,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
M4S,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
M4T,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [55]:
venues_grouped["Total"] = venues_grouped.sum(axis=1)
columns = venues_grouped.columns
venues_grouped.head()

,Venue_categorie_Afghan Restaurant,Venue_categorie_Airport,Venue_categorie_Airport Food Court,Venue_categorie_Airport Gate,Venue_categorie_Airport Lounge,Venue_categorie_Airport Service,Venue_categorie_Airport Terminal,Venue_categorie_American Restaurant,Venue_categorie_Antique Shop,Venue_categorie_Aquarium,...,Venue_categorie_Theme Restaurant,Venue_categorie_Toy / Game Store,Venue_categorie_Trail,Venue_categorie_Train Station,Venue_categorie_Vegetarian / Vegan Restaurant,Venue_categorie_Video Game Store,Venue_categorie_Vietnamese Restaurant,Venue_categorie_Wine Bar,Venue_categorie_Yoga Studio,Total
Postal Code,,,,,,,,,,,,,,,,,,,,,
M4E,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,4
M4K,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,43
M4L,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,19
M4M,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,1,1,37
M4N,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


In [56]:
for col in columns[:-1]:
    venues_grouped[col] = venues_grouped[col]/venues_grouped["Total"]

venues_grouped.head()

,Venue_categorie_Afghan Restaurant,Venue_categorie_Airport,Venue_categorie_Airport Food Court,Venue_categorie_Airport Gate,Venue_categorie_Airport Lounge,Venue_categorie_Airport Service,Venue_categorie_Airport Terminal,Venue_categorie_American Restaurant,Venue_categorie_Antique Shop,Venue_categorie_Aquarium,...,Venue_categorie_Theme Restaurant,Venue_categorie_Toy / Game Store,Venue_categorie_Trail,Venue_categorie_Train Station,Venue_categorie_Vegetarian / Vegan Restaurant,Venue_categorie_Video Game Store,Venue_categorie_Vietnamese Restaurant,Venue_categorie_Wine Bar,Venue_categorie_Yoga Studio,Total
Postal Code,,,,,,,,,,,,,,,,,,,,,
M4E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.000000,0.000000,4
M4K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023256,0.0,0.0,...,0.0,0.0,0.023256,0.0,0.0,0.0,0.0,0.000000,0.023256,43
M4L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,19
M4M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054054,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.027027,0.027027,37
M4N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,3


In [58]:
venues_grouped["Total"] = (venues_grouped["Total"]-venues_grouped["Total"].min())/(venues_grouped["Total"].max()-venues_grouped["Total"].min())
venues_grouped.head()

,Venue_categorie_Afghan Restaurant,Venue_categorie_Airport,Venue_categorie_Airport Food Court,Venue_categorie_Airport Gate,Venue_categorie_Airport Lounge,Venue_categorie_Airport Service,Venue_categorie_Airport Terminal,Venue_categorie_American Restaurant,Venue_categorie_Antique Shop,Venue_categorie_Aquarium,...,Venue_categorie_Theme Restaurant,Venue_categorie_Toy / Game Store,Venue_categorie_Trail,Venue_categorie_Train Station,Venue_categorie_Vegetarian / Vegan Restaurant,Venue_categorie_Video Game Store,Venue_categorie_Vietnamese Restaurant,Venue_categorie_Wine Bar,Venue_categorie_Yoga Studio,Total
Postal Code,,,,,,,,,,,,,,,,,,,,,
M4E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.020408
M4K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023256,0.0,0.0,...,0.0,0.0,0.023256,0.0,0.0,0.0,0.0,0.000000,0.023256,0.418367
M4L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.173469
M4M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054054,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.027027,0.027027,0.357143
M4N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.010204


## Clustering

Now, it's time for clustering. Because we made our researcg on only four boroughs, it can be interessing to see if a 4-cluster will distinguish these four boroughs.

In [60]:
from sklearn.cluster import KMeans

k_means = KMeans(init="k-means++", n_clusters = 4, n_init = 15)
k_means.fit(venues_grouped)
k_means_labels = k_means.labels_
k_means_labels

array([2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       3, 2, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [62]:
venues_grouped["Cluster"] = k_means_labels

In [71]:
column_drop = venues_grouped.columns[:-1]

Index(['Postal Code', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude'], dtype='object')

In [73]:
df_fin = pd.merge(venues_grouped, df_tor, on="Postal Code")

df_fin.drop(columns = column_drop, inplace=True)
df_fin

,Postal Code,Cluster,Borough,Neighbourhood,Latitude,Longitude
0,M4E,2,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,0,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,0,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,0,East Toronto,Studio District,43.659526,-79.340923
4,M4N,2,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,0,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,0,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
7,M4S,0,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,2,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,0,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


We finally obtain our final dataframe. We can now print a map as visualization of these clusters.

In [78]:
color_list = cm.rainbow(np.linspace(0,1,4))
rainbow = [colors.rgb2hex(i) for i in color_list]

tor_map = folium.Map(location = [43.66, -79.39], zoom_start=12)

for lat, lon, code, bor, neigh, cluster in zip(df_fin["Latitude"], df_fin["Longitude"], df_fin["Postal Code"], df_fin["Borough"], df_fin["Neighbourhood"], df_fin["Cluster"]):
    label = folium.Popup(code +',\n '+ bor+ ',\n'+ neigh, parse_html=True)
    folium.CircleMarker([lat, lon], radius =5, popup = label, color = rainbow[cluster], fill =True).add_to(tor_map)

tor_map

Those results seems coherent. Indeed, for instance, it seems normal that all Downtown neighboor share lots of common services.